In [1]:
import numpy as np
import pandas as pd
import pickle
from lbs.dl.metrics import calc_sov
import sklearn.metrics
pd.options.display.float_format = '{:,.3f}'.format
import re

Using TensorFlow backend.


In [2]:
def get_benchmarks(true_column, pred_column):
    yt = []
    yp = []
    sovs = []
    for true, pred in zip(true_column.tolist(), pred_column.tolist()):
        for match in re.finditer(r"([%s]+)" % '1', true):
            sovs.append(calc_sov(true[match.start():match.end()], pred[match.start():match.end()]))
        for y_true, y_pred in zip(true, pred):
            yt.append(y_true)
            yp.append(y_pred)
    prec = sklearn.metrics.precision_score(yt, yp, pos_label='1')
    sens = sklearn.metrics.recall_score(yt, yp, pos_label='1')
    f1 = sklearn.metrics.f1_score(yt, yp, pos_label='1')
    sovs = np.asarray(sovs)
    return(prec, sens, f1, np.mean(sovs), (np.count_nonzero(sovs)/sovs.shape[0]))

In [3]:
mapping = {'deepcoil_assignment': 'DeepCoil', 'cchmmprof_assignment': 'CCHMM_PROF', 'marcoil_10_assignment':'Marcoil_10', 
           'marcoil_50_assignment':'Marcoil_50', 'marcoil_90_assignment':'Marcoil_90', 'pcoils_14_assignment': 'PCoils_14',
           'pcoils_21_assignment': 'PCoils_21', 'pcoils_28_assignment': 'PCoils_28', 'coils_14_assignment': 'Coils_14',
           'coils_21_assignment': 'Coils_21', 'coils_28_assignment': 'Coils_28', 'pcoils_14_assignment_toolkit': 'PCoils_14_Toolkit',
           'pcoils_21_assignment_toolkit': 'PCoils_21_Toolkit', 'pcoils_28_assignment_toolkit':'PCoils_28_Toolkit', 'pcoils_14_nw_assignment_toolkit': 'PCoils_14_NW_Toolkit',
           'pcoils_21_nw_assignment_toolkit': 'PCoils_21_NW_Toolkit', 'pcoils_28_nw_assignment_toolkit':'PCoils_28_NW_Toolkit', 'multicoil2_assignment': 'Multicoil2'}

### Get results from cross-validation

In [4]:
df_cv = pickle.load(open('./../4_Train/out/seq/cv_results.p', 'rb'))

In [5]:
df_cv.shape

(10438, 26)

In [6]:
results = {}
variants = ['deepcoil_assignment', 'cchmmprof_assignment', 'marcoil_10_assignment', 'marcoil_50_assignment',
            'marcoil_90_assignment', 'pcoils_14_assignment', 'pcoils_21_assignment', 'pcoils_28_assignment', 
            'coils_14_assignment', 'coils_21_assignment', 'coils_28_assignment', 'multicoil2_assignment']
for variant in variants:
    results[mapping[variant]] = get_benchmarks(df_cv['socket_assignment'], df_cv[variant])

In [7]:
results_cv = pd.DataFrame.from_dict(results, orient='index')
results_cv.columns = ['Precision', 'Sensitivity', 'F1 score', 'Mean SOV', 'Detected segments']
results_cv.sort_index(inplace=True)

### Get results from test set

In [8]:
df_test = pickle.load(open('./../4_Train/out/seq/test_results.p', 'rb'))
df_test.shape

(1193, 25)

#### Add results from PCoils calculated on Tuebingen Toolkit

In [9]:
df_update = pickle.load(open('./../2_Benchmark_Other_Methods/pcoils/pcoils_test_update.p', 'rb'))
df_test = pd.concat([df_test, df_update], axis=1, sort=True)
df_test.shape

(1193, 28)

In [10]:
results = {}
variants = ['deepcoil_assignment', 'cchmmprof_assignment', 'marcoil_10_assignment', 'marcoil_50_assignment',
            'marcoil_90_assignment', 'pcoils_14_assignment', 'pcoils_21_assignment', 'pcoils_28_assignment',
            'pcoils_14_assignment_toolkit', 'pcoils_21_assignment_toolkit', 'pcoils_28_assignment_toolkit',
            'coils_14_assignment', 'coils_21_assignment', 'coils_28_assignment', 'multicoil2_assignment']
for variant in variants:
    results[mapping[variant]] = get_benchmarks(df_test['socket_assignment'], df_test[variant])

In [11]:
results_test = pd.DataFrame.from_dict(results, orient='index')
results_test.columns = ['Precision', 'Sensitivity', 'F1 score', 'Mean SOV', 'Detected segments']
results_test.sort_index(inplace=True)

### Get results from Li (2016) test set

In [12]:
df_test = pickle.load(open('./../4_Train/out/seq/li2016_results.p', 'rb'))
df_test.shape

(518, 25)

#### Add results from PCoils calculated on Tuebingen Toolkit

In [13]:
df_update = pickle.load(open('./../2_Benchmark_Other_Methods/pcoils/pcoils_li2016_update.p', 'rb'))
df_test = pd.concat([df_test, df_update], axis=1, sort=True)
df_test.shape

(518, 28)

In [14]:
results = {}
variants = ['deepcoil_assignment', 'cchmmprof_assignment', 'marcoil_10_assignment', 'marcoil_50_assignment',
            'marcoil_90_assignment', 'pcoils_14_assignment', 'pcoils_21_assignment', 'pcoils_28_assignment',
            'pcoils_14_assignment_toolkit', 'pcoils_21_assignment_toolkit', 'pcoils_28_assignment_toolkit',
            'coils_14_assignment', 'coils_21_assignment', 'coils_28_assignment', 'multicoil2_assignment']
for variant in variants:
    results[mapping[variant]] = get_benchmarks(df_test['socket_assignment'], df_test[variant])

In [15]:
results_li2016 = pd.DataFrame.from_dict(results, orient='index')
results_li2016.columns = ['Precision', 'Sensitivity', 'F1 score', 'Mean SOV', 'Detected segments']
results_li2016.sort_index(inplace=True)

### Show results

#### Cross-validation

In [16]:
results_cv

,Precision,Sensitivity,F1 score,Mean SOV,Detected segments
CCHMM_PROF,0.382,0.227,0.284,0.147,0.148
Coils_14,0.241,0.259,0.249,0.236,0.290
Coils_21,0.336,0.274,0.302,0.213,0.238
Coils_28,0.400,0.274,0.325,0.188,0.203
DeepCoil,0.384,0.450,0.414,0.394,0.518
Marcoil_10,0.284,0.369,0.321,0.271,0.293
Marcoil_50,0.400,0.261,0.316,0.175,0.191
Marcoil_90,0.491,0.176,0.259,0.111,0.122
Multicoil2,0.546,0.195,0.287,0.102,0.104
PCoils_14,0.382,0.235,0.291,0.192,0.226


#### Test set

In [17]:
results_test

,Precision,Sensitivity,F1 score,Mean SOV,Detected segments
CCHMM_PROF,0.255,0.310,0.280,0.234,0.237
Coils_14,0.252,0.306,0.276,0.261,0.309
Coils_21,0.332,0.322,0.327,0.228,0.252
Coils_28,0.386,0.317,0.348,0.199,0.218
DeepCoil,0.390,0.491,0.435,0.413,0.518
Marcoil_10,0.290,0.427,0.345,0.296,0.319
Marcoil_50,0.392,0.324,0.355,0.200,0.223
Marcoil_90,0.482,0.237,0.318,0.135,0.151
Multicoil2,0.550,0.242,0.336,0.124,0.127
PCoils_14,0.409,0.290,0.340,0.203,0.230


#### Li (2016)

In [18]:
results_li2016

,Precision,Sensitivity,F1 score,Mean SOV,Detected segments
CCHMM_PROF,0.445,0.257,0.326,0.192,0.192
Coils_14,0.446,0.260,0.328,0.255,0.311
Coils_21,0.512,0.266,0.350,0.232,0.259
Coils_28,0.559,0.278,0.372,0.219,0.239
DeepCoil,0.607,0.465,0.527,0.424,0.538
Marcoil_10,0.485,0.400,0.439,0.333,0.358
Marcoil_50,0.540,0.263,0.354,0.209,0.227
Marcoil_90,0.578,0.146,0.233,0.111,0.121
Multicoil2,0.610,0.170,0.265,0.098,0.101
PCoils_14,0.542,0.239,0.332,0.218,0.256
